In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from torchvision import datasets, transforms
import torch 

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, )), ])

train = datasets.FashionMNIST('FashionMNIST/data', train=True, download=True, transform=transform)
training_data = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

test = datasets.FashionMNIST('FashionMNIST/data', train=False, download=True, transform=transform)
testing_data = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)



In [3]:
import torch.nn as nn 
import torch.nn.functional as F

input_size = 784
hidden_size = [128, 64]
output_size = 10

#MODEL IN MAIN
model = nn.Sequential(nn.Linear(input_size, hidden_size[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_size[0], hidden_size[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_size[1], output_size),
                      nn.LogSoftmax(dim=1))

model


Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax()
)

In [0]:
class Net(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 10)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.log_softmax(self.fc4(x), dim=1)
  
    return x


In [0]:
def training_function(LR):
  import torch.optim as optim 
  optimizer = optim.SGD(model.parameters(), lr=LR)
  criterion = nn.NLLLoss()
  EPOCHS = 5
  loss = None

  for epoch in range(EPOCHS):
    total_loss = 0
    for images, labels in training_data:
      # images = images.view(-1, 784)
      outputs = model(images.view(-1, 784)) 
      loss = criterion(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      total_loss += loss.item()
      optimizer.step()
    
    print("Loss of epoch %d : %.4f" %(epoch+1, total_loss/len(training_data)))

In [0]:
def testing_function(model):
  correct_label = 0
  total = 0
  with torch.no_grad():
    for images, labels in testing_data:
      # images = images.view(-1, 784)
      outputs = model(images.view(-1, 784))
      for i in range(len(labels)):
        if torch.argmax(outputs[i]) == labels[i]:
          correct_label += 1
        total += 1
  print("Accuracy in test_data %.2f %%." %(100*correct_label/total))
  print("------------------------------------------------------------")

In [7]:
#TRAIN MODEL IN MAIN
training_function(0.01)
#CALCULATE ACCURACY IN TEST CASE
testing_function(model)



Loss of epoch 1 : 1.0004
Loss of epoch 2 : 0.5548
Loss of epoch 3 : 0.4941
Loss of epoch 4 : 0.4603
Loss of epoch 5 : 0.4361
Accuracy in test_data 83.58 %.
------------------------------------------------------------


In [0]:
def training_multi_function(LR):
  import torch.optim as optim 
  model = Net()
  optimizer = optim.SGD(model.parameters(), lr=LR)
  criterion = nn.NLLLoss()
  EPOCHS = 5
  loss = None
  for epoch in range(EPOCHS):
    total_loss = 0
    for images, labels in training_data:
      # images = images.view(-1, 784)
      outputs = model(images.view(-1, 784)) 
      loss = criterion(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      total_loss += loss.item()
      optimizer.step()
    
    print("Loss of epoch %d : %.4f" %(epoch+1, total_loss/len(training_data)))
  testing_function(model)

In [9]:
#TEST SOME LEARNING RATE WITH MODEL IN CLASS
learning_rate = [0.01, 1, 0.003, 0.1, 0.05]
for lr in learning_rate:
  print("Case learning rate is %.3f" %lr)
  training_multi_function(lr)
  





Case learning rate is 0.010
Loss of epoch 1 : 1.3092
Loss of epoch 2 : 0.6080
Loss of epoch 3 : 0.5234
Loss of epoch 4 : 0.4775
Loss of epoch 5 : 0.4496
Accuracy in test_data 83.25 %.
------------------------------------------------------------
Case learning rate is 1.000
Loss of epoch 1 : 5.1551
Loss of epoch 2 : 2.3063
Loss of epoch 3 : 2.3061
Loss of epoch 4 : 2.3062
Loss of epoch 5 : 2.3065
Accuracy in test_data 10.00 %.
------------------------------------------------------------
Case learning rate is 0.003
Loss of epoch 1 : 2.0959
Loss of epoch 2 : 1.2342
Loss of epoch 3 : 0.8132
Loss of epoch 4 : 0.6858
Loss of epoch 5 : 0.6240
Accuracy in test_data 76.74 %.
------------------------------------------------------------
Case learning rate is 0.100
Loss of epoch 1 : 0.6456
Loss of epoch 2 : 0.4191
Loss of epoch 3 : 0.3697
Loss of epoch 4 : 0.3396
Loss of epoch 5 : 0.3171
Accuracy in test_data 86.82 %.
------------------------------------------------------------
Case learning rate i